In [ ]:
from datetime import datetime

import pandas as pd
from PIL import Image

from fine_tuning.speciesnet_head.speciesnet_polish_model import get_model, predict_batch

In [ ]:
CROP_SIZE = 480 # change and check?
BATCH_SIZE = 20
classifier, class_names = get_model()
classifier.to('cuda')

images = pd.read_csv('megadetector_results.csv', index_col=0)
results = pd.DataFrame({'image': [], 'detected_animal': [], 'confidence': []})

In [ ]:
def crop_normalized_bbox(img: Image.Image, bbox: list[float]):
    """
    img: PIL.Image opened image
    bbox: list [x, y, w, h], normalized 0-1
    returns cropped PIL.Image
    """
    W, H = img.size
    x, y, w, h = bbox

    left   = int(x * W)
    top    = int(y * H)
    right  = int((x + w) * W)
    bottom = int((y + h) * H)

    # TODO maybe square?

    return img.crop((left, top, right, bottom))

In [8]:
currently_classifying = images['image_path'][0].rsplit('/')[2]
print('Currently: ', currently_classifying)
batch = []
paths = []

for _, row in images.iterrows():
    # just progress info
    image_path = row['image_path']
    if currently_classifying != image_path.rsplit('/')[2]:
        currently_classifying = image_path.rsplit('/')[2]
        print('Currently: ' + currently_classifying)

    # only animals
    category = row['category']
    if category != 1:
        results.loc[len(results)] = [image_path, 'empty', 0]
        continue

    # image
    try:
        image = Image.open(image_path).convert("RGB")
        cropped_image = crop_normalized_bbox(image, row['bbox'])
    except Exception as e:
        print(f'Error in image: {e}')
        continue

    paths.append(image_path)
    batch.append(cropped_image)

    # run classifier every N images (e.g. 32)
    if len(batch) == BATCH_SIZE:
        preds = predict_batch(classifier, batch, class_names)
        print(preds)
        # TODO add to results
        # if confidence less than threshold: other
        batch = []
        paths = []

now = datetime.now().strftime('%Y_%m_%d_%H_%M')
results.to_csv('results/results_speciesnet_head_' + now + '.csv')

SyntaxError: expected 'except' or 'finally' block (1979548941.py, line 15)

In [ ]:
def cropSquareCVtoPIL(imagecv, box):
    x1, y1, x2, y2 = box
    xsize = (x2-x1)
    ysize = (y2-y1)
    if xsize>ysize:
        y1 = y1-int((xsize-ysize)/2)
        y2 = y2+int((xsize-ysize)/2)
    if ysize>xsize:
        x1 = x1-int((ysize-xsize)/2)
        x2 = x2+int((ysize-xsize)/2)
    height, width, _ = imagecv.shape
    croppedimagecv = imagecv[max(0,int(y1)):min(int(y2),height),max(0,int(x1)):min(int(x2),width)]
    croppedimage = Image.fromarray(croppedimagecv[:,:,(2,1,0)]) # converted to PIL BGR image
    return croppedimage

### Deers thinning

In [26]:
import numpy as np

In [34]:
images = pd.read_csv('y_clean.csv', index_col=0)
images.RelativePath = images.RelativePath.str.replace('\\', '/')

In [36]:
mask_roedeer = images["species"] == "roedeer"
keep_mask = np.random.rand(mask_roedeer.sum()) < 0.3
final_mask = ~mask_roedeer.copy()
final_mask[mask_roedeer] = keep_mask

images_thinned = images[final_mask].reset_index(drop=True)

In [38]:
mask_reddeer = images_thinned["species"] == "reddeer"
keep_mask = np.random.rand(mask_reddeer.sum()) < 0.2
final_mask = ~mask_reddeer.copy()
final_mask[mask_reddeer] = keep_mask

images_thinned = images_thinned[final_mask].reset_index(drop=True)

In [39]:
images[images.species == 'reddeer']

,File,RelativePath,species
6,2023-07-18 02-03-23.JPG,01_CZARNE/B/Lato/2,reddeer
7,2023-07-18 02-03-30.JPG,01_CZARNE/B/Lato/2,reddeer
8,2023-07-18 02-03-38.JPG,01_CZARNE/B/Lato/2,reddeer
9,2023-07-18 02-03-54.JPG,01_CZARNE/B/Lato/2,reddeer
10,2023-07-18 02-06-43.JPG,01_CZARNE/B/Lato/2,reddeer
...,...,...,...
33291,2023-03-06 13-13-42.JPG,30_SUCHEDNIÓW/R/Zima/2,reddeer
33292,2023-03-06 13-13-48.JPG,30_SUCHEDNIÓW/R/Zima/2,reddeer
33293,2023-03-06 13-13-55.JPG,30_SUCHEDNIÓW/R/Zima/2,reddeer
33294,2023-03-06 13-14-11.JPG,30_SUCHEDNIÓW/R/Zima/2,reddeer


In [40]:
images_thinned[images_thinned.species == 'reddeer']

,File,RelativePath,species
6,2023-07-18 02-03-54.JPG,01_CZARNE/B/Lato/2,reddeer
7,2023-07-18 02-06-43.JPG,01_CZARNE/B/Lato/2,reddeer
9,2023-07-26 03-47-00.JPG,01_CZARNE/B/Lato/2,reddeer
10,2023-07-26 03-47-13.JPG,01_CZARNE/B/Lato/2,reddeer
11,2023-07-28 19-09-08.JPG,01_CZARNE/B/Lato/2,reddeer
...,...,...,...
18079,2023-05-31 03-59-22.JPG,30_SUCHEDNIÓW/R/Wiosna/2,reddeer
18090,2022-05-20 08-36-20.JPG,30_SUCHEDNIÓW/R/Wiosna/3,reddeer
18091,2022-05-20 08-37-05.JPG,30_SUCHEDNIÓW/R/Wiosna/3,reddeer
18092,2022-05-20 08-38-02.JPG,30_SUCHEDNIÓW/R/Wiosna/3,reddeer


In [41]:
images[images.species == 'roedeer']

,File,RelativePath,species
83,2022-12-06 21-20-18.JPG,01_CZARNE/B/Późna jesień/3,roedeer
84,2022-12-06 21-20-43.JPG,01_CZARNE/B/Późna jesień/3,roedeer
203,2023-05-13 05-58-53.JPG,01_CZARNE/B/Wiosna/2,roedeer
211,2023-05-05 09-06-50.JPG,01_CZARNE/B/Wiosna/3,roedeer
212,2023-05-05 09-07-14.JPG,01_CZARNE/B/Wiosna/3,roedeer
...,...,...,...
33286,2023-03-03 21-24-55.JPG,30_SUCHEDNIÓW/R/Zima/2,roedeer
33287,2023-03-03 21-25-05.JPG,30_SUCHEDNIÓW/R/Zima/2,roedeer
33288,2023-03-04 12-32-12.JPG,30_SUCHEDNIÓW/R/Zima/2,roedeer
33289,2023-03-04 12-32-20.JPG,30_SUCHEDNIÓW/R/Zima/2,roedeer


In [42]:
images_thinned[images_thinned.species == 'roedeer']

,File,RelativePath,species
37,2022-12-06 21-20-43.JPG,01_CZARNE/B/Późna jesień/3,roedeer
78,2023-05-13 05-58-53.JPG,01_CZARNE/B/Wiosna/2,roedeer
82,2023-05-05 09-11-05.JPG,01_CZARNE/B/Wiosna/3,roedeer
83,2023-05-06 01-13-02.JPG,01_CZARNE/B/Wiosna/3,roedeer
88,2023-05-09 13-42-13.JPG,01_CZARNE/B/Wiosna/3,roedeer
...,...,...,...
18094,2022-06-02 15-07-28.JPG,30_SUCHEDNIÓW/R/Wiosna/3,roedeer
18095,2022-06-08 07-46-13.JPG,30_SUCHEDNIÓW/R/Wiosna/3,roedeer
18098,2023-02-26 17-05-57.JPG,30_SUCHEDNIÓW/R/Zima/2,roedeer
18099,2023-02-26 17-06-05.JPG,30_SUCHEDNIÓW/R/Zima/2,roedeer


In [43]:
images_thinned.to_csv('y_clean_thin.csv')

### Just paths

In [68]:
images = pd.read_csv('y_clean_thin.csv', index_col=0)

In [71]:
images['image_path'] = '../pictures/' + images['RelativePath'] + '/' + images['File']
images.to_csv('y_clean_thin.csv')